In [7]:
import pandas as pd
import numpy as np
import re
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify
from tqdm import tqdm
import datetime
import time

## Preprocessing initial ipynb

In [2]:
df = pd.read_csv('merged_dataset.csv')

In [3]:
df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,0.904,0.8130,...,0.03110,0.006970,0.0471,0.8100,125.461,4.0,2005-07-04,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,то,0,0.774,0.8380,...,0.02490,0.025000,0.2420,0.9240,143.040,4.0,2003-11-13,83.0,83.0,dance_pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.7590,...,0.00238,0.000000,0.0598,0.7010,99.252,4.0,2003-06-23,15.0,87.0,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,0.892,0.7140,...,0.20100,0.000234,0.0521,0.8170,100.972,4.0,2002-11-04,78.0,83.0,dance_pop pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,0.853,0.6060,...,0.05610,0.000000,0.3130,0.6540,94.759,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,302981,0.540,0.0429,...,0.61400,0.880000,0.1110,0.1860,115.493,4.0,1998,0.0,36.0,scottish_indie slowcore
2262288,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops,302985,0.674,0.4140,...,0.96200,0.671000,0.1730,0.8490,121.145,4.0,2004-01-01,16.0,14.0,military_band
2262289,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago,302998,0.637,0.4830,...,0.60800,0.000000,0.3050,0.5620,69.964,4.0,2010-01-01,14.0,64.0,modern_rock neo_mellow neon_pop_punk pop_punk ...
2262290,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought,302998,0.557,0.6100,...,0.00493,0.000059,0.1680,0.0848,113.939,4.0,2016-11-07,0.0,46.0,canadian_hip_hop underground_hip_hop


In [4]:
df.isnull().sum()

artist_name            10
track_uri               0
artist_uri              0
track_name              7
album_uri               0
duration_ms             0
album_name              7
album_id                0
danceability        53154
energy              53154
key                 53154
loudness            53154
mode                53154
speechiness         53154
acousticness        53154
instrumentalness    53154
liveness            53154
valence             53154
tempo               53154
time_signature      53154
release_date        53154
popularity          53154
artist_pop          53154
genres              53154
dtype: int64

- видаляю: artist_name(10), track_name(7), album_name(7), все інше там де 53154

In [5]:
df = df.dropna()
df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,0.904,0.8130,...,0.03110,0.006970,0.0471,0.8100,125.461,4.0,2005-07-04,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,то,0,0.774,0.8380,...,0.02490,0.025000,0.2420,0.9240,143.040,4.0,2003-11-13,83.0,83.0,dance_pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.7590,...,0.00238,0.000000,0.0598,0.7010,99.252,4.0,2003-06-23,15.0,87.0,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,0.892,0.7140,...,0.20100,0.000234,0.0521,0.8170,100.972,4.0,2002-11-04,78.0,83.0,dance_pop pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,0.853,0.6060,...,0.05610,0.000000,0.3130,0.6540,94.759,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,302981,0.540,0.0429,...,0.61400,0.880000,0.1110,0.1860,115.493,4.0,1998,0.0,36.0,scottish_indie slowcore
2262288,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops,302985,0.674,0.4140,...,0.96200,0.671000,0.1730,0.8490,121.145,4.0,2004-01-01,16.0,14.0,military_band
2262289,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago,302998,0.637,0.4830,...,0.60800,0.000000,0.3050,0.5620,69.964,4.0,2010-01-01,14.0,64.0,modern_rock neo_mellow neon_pop_punk pop_punk ...
2262290,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought,302998,0.557,0.6100,...,0.00493,0.000059,0.1680,0.0848,113.939,4.0,2016-11-07,0.0,46.0,canadian_hip_hop underground_hip_hop


In [6]:
df.isnull().sum()

artist_name         0
track_uri           0
artist_uri          0
track_name          0
album_uri           0
duration_ms         0
album_name          0
album_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
release_date        0
popularity          0
artist_pop          0
genres              0
dtype: int64

In [7]:
df['release_date'] = df['release_date'].astype(str).apply(lambda x: re.search(r'\d{4}', x).group(0) if re.search(r'\d{4}', x) else None)
df['release_date']

/var/folders/nd/7lptb8w57_g497n2mc741rmc0000gn/T/ipykernel_5364/2463670836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = df['release_date'].astype(str).apply(lambda x: re.search(r'\d{4}', x).group(0) if re.search(r'\d{4}', x) else None)


0          2005
1          2003
2          2003
3          2002
4          2000
           ... 
2262287    1998
2262288    2004
2262289    2010
2262290    2016
2262291    2014
Name: release_date, Length: 2209138, dtype: object

In [8]:
unique_dates = df['release_date'].dropna().unique()
print("Унікальні дати (роки):")
print(unique_dates)

Унікальні дати (роки):
['2005' '2003' '2002' '2000' '2004' '1999' '2006' '2008' '2011' '2007'
 '2010' '2009' '2013' '1982' '2015' '2012' '1971' '1994' '1968' '1969'
 '1973' '1976' '1978' '1975' '1974' '1987' '1984' '1981' '2016' '2017'
 '2014' '1988' '1985' '1995' '1992' '1996' '1997' '1989' '1986' '1990'
 '1970' '1993' '1960' '1998' '2001' '1963' '1977' '1965' '1972' '1991'
 '1900' '1979' '1967' '1980' '1942' '1961' '1957' '1948' '1964' '1958'
 '1962' '1966' '1983' '1959' '0000' '1929' '1956' '1935' '1947' '1946'
 '1955' '1951' '1953' '1954' '1952' '1933' '2022' '1937' '1939' '1945'
 '1950' '1936' '1901' '2020' '1930' '1949' '2021' '2018' '1928' '1927'
 '1923' '1925' '2019' '1941' '2023' '1885' '1932' '1926' '1940' '1938'
 '1944' '2024' '1919' '1934' '0013' '1924' '1943' '1197' '1899' '1905'
 '1909' '1931' '1920' '1917' '1918' '1908' '0001' '1911' '1906' '1888']


In [9]:
drop_dates = ['0000', '0013', '0001', '1197']
df = df[~df['release_date'].isin(drop_dates)]
print("Після видалення:")
print(df['release_date'].unique())

Після видалення:
['2005' '2003' '2002' '2000' '2004' '1999' '2006' '2008' '2011' '2007'
 '2010' '2009' '2013' '1982' '2015' '2012' '1971' '1994' '1968' '1969'
 '1973' '1976' '1978' '1975' '1974' '1987' '1984' '1981' '2016' '2017'
 '2014' '1988' '1985' '1995' '1992' '1996' '1997' '1989' '1986' '1990'
 '1970' '1993' '1960' '1998' '2001' '1963' '1977' '1965' '1972' '1991'
 '1900' '1979' '1967' '1980' '1942' '1961' '1957' '1948' '1964' '1958'
 '1962' '1966' '1983' '1959' '1929' '1956' '1935' '1947' '1946' '1955'
 '1951' '1953' '1954' '1952' '1933' '2022' '1937' '1939' '1945' '1950'
 '1936' '1901' '2020' '1930' '1949' '2021' '2018' '1928' '1927' '1923'
 '1925' '2019' '1941' '2023' '1885' '1932' '1926' '1940' '1938' '1944'
 '2024' '1919' '1934' '1924' '1943' '1899' '1905' '1909' '1931' '1920'
 '1917' '1918' '1908' '1911' '1906' '1888']


In [10]:
df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,0.904,0.8130,...,0.03110,0.006970,0.0471,0.8100,125.461,4.0,2005,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,то,0,0.774,0.8380,...,0.02490,0.025000,0.2420,0.9240,143.040,4.0,2003,83.0,83.0,dance_pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.7590,...,0.00238,0.000000,0.0598,0.7010,99.252,4.0,2003,15.0,87.0,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,0.892,0.7140,...,0.20100,0.000234,0.0521,0.8170,100.972,4.0,2002,78.0,83.0,dance_pop pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,0.853,0.6060,...,0.05610,0.000000,0.3130,0.6540,94.759,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,302981,0.540,0.0429,...,0.61400,0.880000,0.1110,0.1860,115.493,4.0,1998,0.0,36.0,scottish_indie slowcore
2262288,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops,302985,0.674,0.4140,...,0.96200,0.671000,0.1730,0.8490,121.145,4.0,2004,16.0,14.0,military_band
2262289,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago,302998,0.637,0.4830,...,0.60800,0.000000,0.3050,0.5620,69.964,4.0,2010,14.0,64.0,modern_rock neo_mellow neon_pop_punk pop_punk ...
2262290,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought,302998,0.557,0.6100,...,0.00493,0.000059,0.1680,0.0848,113.939,4.0,2016,0.0,46.0,canadian_hip_hop underground_hip_hop


In [11]:
unique_genres = df['genres'].str.split(expand=True).stack().value_counts()
unique_genres

unknown                  370809
rock                      72212
pop                       62560
classical                 53229
classic_rock              41831
                          ...  
modern_j-rock                 1
metal_cearense                1
haur_kantak                   1
dragspel                      1
vietnamese_electronic         1
Length: 5985, dtype: int64

In [12]:
russian_genres = unique_genres[unique_genres.index.str.contains('russian', case=False)]
print(russian_genres)


russian_romanticism                5022
russian_pop                        2941
russian_modern_classical           2275
classic_russian_pop                1802
russian_rock                       1233
russian_hip_hop                    1176
classic_russian_rock                754
russian_edm                         568
russian_trance                      544
russian_dance                       441
russian_gangster_rap                411
russian_punk                        342
russian_electronic                  290
russian_trap                        272
russian_chanson                     259
russian_folk                        218
russian_alternative_rock            216
russian_metal                       206
russian_classical_piano             170
russian_dance_pop                   166
russian_alt_pop                     153
russian_contemporary_classical      148
russian_dnb                         147
russian_romance                     145
russian_orchestra                   133


In [13]:
df = df[~df['genres'].str.contains('russian', case=False, na=False)]
print(df.head())

         artist_name                             track_uri  \
0      Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
1     Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
2            Beyoncé  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
3  Justin Timberlake  spotify:track:1AWQoqb9bSvzTjaLralEkT   
4             Shaggy  spotify:track:1lzr43nnXAijIGYnCT8M8H   

                              artist_uri  \
0  spotify:artist:2wIVse2owClT7go1WT98tk   
1  spotify:artist:26dSoYclwsYLMAKD3tpOr4   
2  spotify:artist:6vWDO969PvNqNYHIOW5v0m   
3  spotify:artist:31TPClRtHm23RisEBtV3X7   
4  spotify:artist:5EvFsr3kj42KNv97ZEnqij   

                                   track_name  \
0  Lose Control (feat. Ciara & Fat Man Scoop)   
1                                       Toxic   
2                               Crazy In Love   
3                              Rock Your Body   
4                                It Wasn't Me   

                              album_uri  duration_ms  \
0  spotify:

In [14]:
unique_genres = df['genres'].str.split(expand=True).stack().value_counts()
unique_genres

unknown            370809
rock                72212
pop                 62560
classical           46448
classic_rock        41831
                    ...  
rap_congolais           1
quarteto_gospel         1
bells                   1
mluvene_slovo           1
zampogna                1
Length: 5923, dtype: int64

In [15]:
russian_genres = unique_genres[unique_genres.index.str.contains('russian', case=False)]
print(russian_genres)

Series([], dtype: int64)


In [16]:
df.shape

(2191397, 24)

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'left')
unknown_genres = df[df['genres'] == 'unknown']
unknown_genres['year'] = unknown_genres['release_date'].dropna().apply(lambda x: str(x)[:4])
songs_per_year = unknown_genres['year'].value_counts().sort_index()
print("Кількість пісень з жанром 'unknown' по роках:")
print(songs_per_year)


Кількість пісень з жанром 'unknown' по роках:
1900       35
1905        5
1906        1
1908        1
1909        2
1920        1
1925        1
1928        9
1930        1
1932        6
1933        8
1934        3
1935        1
1936       11
1937        5
1938        4
1939        1
1940        3
1941       15
1942        7
1943        1
1945        3
1946        9
1947        4
1949       21
1950       19
1951       14
1952        4
1953       38
1954       40
1955       33
1956       96
1957       74
1958       92
1959      109
1960      157
1961      100
1962       73
1963      276
1964      180
1965       87
1966      114
1967      131
1968      358
1969      222
1970      169
1971      254
1972      196
1973      260
1974      159
1975      310
1976      197
1977      262
1978      396
1979      281
1980      237
1981      235
1982      257
1983      196
1984      277
1985      321
1986      379
1987      442
1988      459
1989      446
1990      796
1991      813
1992     1100
19

In [19]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'left')
songs_per_date = df['release_date'].dropna().value_counts().sort_index()

print("Кількість пісень по унікальній даті:")
print(songs_per_date)

Кількість пісень по унікальній даті:
1888         2
1899         1
1900       532
1901         4
1905        24
1906         2
1908         1
1909         5
1911         3
1917        25
1918         2
1919         2
1920         2
1923        51
1924         5
1925        33
1926        45
1927        30
1928        57
1929        56
1930        45
1931         9
1932        29
1933       119
1934        12
1935       116
1936       107
1937        68
1938        35
1939       125
1940        49
1941        92
1942       683
1943        10
1944        25
1945        72
1946       324
1947       112
1948       294
1949       165
1950       288
1951       365
1952       174
1953       352
1954       477
1955       925
1956      1934
1957      2240
1958      1934
1959      2710
1960      2378
1961      2523
1962      2363
1963      2968
1964      2992
1965      3545
1966      3103
1967      4122
1968      4466
1969      4878
1970      5861
1971      4838
1972      5585
1973      5645
197

In [20]:
print(df['release_date'].dtype)

object


In [21]:
df['release_date'] = pd.to_numeric(df['release_date'], errors='coerce')
df = df[(df['release_date'] >= 1955) & (df['release_date'] <= 2024)]
df['release_date'] = df['release_date'].astype('int16')

In [22]:
print(df.shape)
df.head(4)

(2186368, 24)


,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,0.904,0.813,4.0,-7.105,0.0,0.121,0.03110,0.006970,0.0471,0.810,125.461,4.0,2005,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,то,0,0.774,0.838,5.0,-3.914,0.0,0.114,0.02490,0.025000,0.2420,0.924,143.040,4.0,2003,83.0,83.0,dance_pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.759,2.0,-6.583,0.0,0.209,0.00238,0.000000,0.0598,0.701,99.252,4.0,2003,15.0,87.0,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,0.892,0.714,4.0,-6.055,0.0,0.141,0.20100,0.000234,0.0521,0.817,100.972,4.0,2002,78.0,83.0,dance_pop pop


In [23]:
songs_per_date = df['release_date'].dropna().value_counts().sort_index()

print("Кількість пісень по унікальній даті:")
print(songs_per_date)

Кількість пісень по унікальній даті:
1955       925
1956      1934
1957      2240
1958      1934
1959      2710
1960      2378
1961      2523
1962      2363
1963      2968
1964      2992
1965      3545
1966      3103
1967      4122
1968      4466
1969      4878
1970      5861
1971      4838
1972      5585
1973      5645
1974      4412
1975      5248
1976      4950
1977      5465
1978      5581
1979      5682
1980      6022
1981      5306
1982      5917
1983      5784
1984      6242
1985      6460
1986      7092
1987      8098
1988      8709
1989      9170
1990     11725
1991     13733
1992     13249
1993     15884
1994     19222
1995     21088
1996     19945
1997     22598
1998     24289
1999     28013
2000     30225
2001     32812
2002     33488
2003     39439
2004     42978
2005     54107
2006     63795
2007     69592
2008     79032
2009     93190
2010    111934
2011    138630
2012    156011
2013    165691
2014    182644
2015    200638
2016    195135
2017    141620
2018       206
201

In [24]:
song_count = df.groupby('album_id').size().reset_index(name='song_count')
song_count_dict = song_count.set_index('album_id')['song_count'].to_dict()
song_count_dict

{0: 51,
 1: 39,
 2: 62,
 3: 125,
 4: 17,
 5: 72,
 6: 16,
 7: 52,
 8: 46,
 9: 20,
 10: 72,
 11: 128,
 12: 9,
 13: 13,
 14: 95,
 15: 7,
 16: 102,
 17: 73,
 18: 67,
 19: 83,
 20: 14,
 21: 14,
 22: 39,
 23: 20,
 24: 7,
 25: 38,
 26: 24,
 27: 11,
 28: 75,
 29: 31,
 30: 145,
 31: 91,
 32: 95,
 33: 16,
 34: 68,
 35: 55,
 36: 9,
 37: 67,
 38: 40,
 39: 55,
 40: 56,
 41: 184,
 42: 116,
 43: 17,
 44: 28,
 45: 49,
 46: 11,
 47: 14,
 48: 20,
 49: 45,
 50: 35,
 51: 33,
 52: 13,
 53: 93,
 54: 5,
 55: 108,
 56: 18,
 57: 19,
 58: 37,
 59: 111,
 60: 53,
 61: 6,
 62: 20,
 63: 32,
 64: 86,
 65: 110,
 66: 74,
 67: 8,
 68: 152,
 69: 16,
 70: 69,
 71: 12,
 72: 189,
 73: 23,
 74: 5,
 75: 13,
 76: 25,
 77: 21,
 78: 14,
 79: 16,
 80: 51,
 81: 34,
 82: 25,
 83: 52,
 84: 28,
 85: 35,
 86: 26,
 87: 24,
 88: 58,
 89: 85,
 90: 95,
 91: 58,
 92: 6,
 93: 40,
 94: 81,
 95: 133,
 96: 18,
 97: 224,
 98: 76,
 99: 42,
 100: 21,
 101: 19,
 102: 40,
 103: 159,
 104: 51,
 105: 70,
 106: 98,
 107: 22,
 108: 49,
 109: 124,
 110

In [25]:
songs_2024 = df[df['release_date'] == 2024]
print(songs_2024)


        artist_name                     track_uri                             artist_uri                             track_name                                         album_uri                              duration_ms album_name                                          album_id  danceability  energy  key   loudness  mode  speechiness  acousticness  instrumentalness  liveness  valence  tempo    time_signature  release_date  popularity  artist_pop genres                               
294272          L'Orchestra Cinematique  spotify:track:6cwQORvjzirSEy6kAK3gVy  spotify:artist:4m1i53DaEliWzSkSFwqv5W  Sharks don't Sleep (From the "Captain America:...  spotify:album:0yHSOFNOkcQBKOnwFQzsXM  117693       Sharks don't Sleep (From the "Captain America:...   94737    0.368         0.390    4.0 -14.422    0.0   0.0466       0.54100       0.90400           0.1230    0.0879   120.009  4.0             2024          0.0         52.0                                   scorecore
294291          L'Orch

In [26]:
import re
def contains_cyrillic(text):
    return bool(re.search(r'[а-яА-ЯёЁ]', str(text)))
russian_songs = df[df['track_name'].apply(contains_cyrillic)]
df = df[~df['track_name'].apply(contains_cyrillic)]
df = df[~df['artist_name'].apply(contains_cyrillic)]
df.shape

(2184530, 24)

In [36]:
stop_list = ['Andrei Krylov', 'Sergei Rachmaninoff', 'St. Petersburg Orchestra of the State Hermitage Museum Camerata', 
             'St. Petersburg Festival Symphony Orchestra',
             'St.Petersburg Orchestra Of The State Hermitage Museum Camerata', 'Saint Petersburg Radio and TV Symphony Orchestra',
             'St. Petersburg Chamber Orchestra Canon', 'St.Petersburg Orchestra Of The State Hermitage Museum Camerata', 'St. Petersburg Chamber Orchestra Canon',
             'Chamber Ensemble Soloists of St. Petersburg', 'St-Petersburg Ska-Jazz Review', 'St. Petersburg Chamber Choir', 'Quartet Of Soloists Of Camerata St. Petersburg'
             'Saint Petersburg Radio and TV Symphony Orchestra', 'Dmitri Hvorostovsky','DJ RusLan', 'Regina Spektor']

In [28]:
df = df[~df['artist_name'].str.contains('Petersburg', case=False, na=False)]
print(df.head())

  artist_name        track_uri                             artist_uri                             track_name                                  album_uri                              duration_ms album_name                                     album_id  danceability  energy  key  loudness  mode  speechiness  acousticness  instrumentalness  liveness  valence  tempo    time_signature  release_date  popularity  artist_pop genres                                            
0      Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI  spotify:artist:2wIVse2owClT7go1WT98tk  Lose Control (feat. Ciara & Fat Man Scoop)  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K  226863                                       The Cookbook  0         0.904         0.813   4.0 -7.105     0.0   0.1210       0.03110       0.006970          0.0471    0.810    125.461  4.0             2005          65.0        72.0        dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1     Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak  

In [29]:
df = df[~df['artist_name'].str.contains('Petersburg', case=False, na=False)]
print(df.head())

  artist_name        track_uri                             artist_uri                             track_name                                  album_uri                              duration_ms album_name                                     album_id  danceability  energy  key  loudness  mode  speechiness  acousticness  instrumentalness  liveness  valence  tempo    time_signature  release_date  popularity  artist_pop genres                                            
0      Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI  spotify:artist:2wIVse2owClT7go1WT98tk  Lose Control (feat. Ciara & Fat Man Scoop)  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K  226863                                       The Cookbook  0         0.904         0.813   4.0 -7.105     0.0   0.1210       0.03110       0.006970          0.0471    0.810    125.461  4.0             2005          65.0        72.0        dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1     Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak  

In [30]:
keywords = ['Russian', 'Soviet', 'Putin', 'USSR', 'Siberia']
df = df[~df.apply(lambda row: row.astype(str).str.contains('|'.join(keywords), case=False, na=False).any(), axis=1)]

In [31]:
df = df[~df.apply(lambda row: row.astype(str).str.contains('Russia', case=False).any(), axis=1)]

In [32]:
df = df[~df.apply(lambda row: row.astype(str).str.contains('Moscow', case=False).any(), axis=1)]

In [33]:
df.shape

(2181533, 24)

In [34]:
df.to_csv("processed_dataset_VD.csv", index=False)

In [36]:
df.shape

(2181533, 24)

## Merging with spotify_songs.csv

In [50]:
df = pd.read_csv("processed_dataset_VD.csv")

In [51]:
to_merged_df = pd.read_csv("spotify_songs.csv")
to_merged_df

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432


In [52]:
df.columns

Index(['artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
       'duration_ms', 'album_name', 'album_id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'release_date', 'popularity', 'artist_pop', 'genres'],
      dtype='object')

In [53]:
to_merged_df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In [54]:
set_1 = {'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
         'duration_ms', 'album_name', 'album_id', 'danceability', 'energy',
         'key', 'loudness', 'mode', 'speechiness', 'acousticness',
         'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
         'release_date', 'popularity', 'artist_pop', 'genres'}

set_2 = {'track_id', 'track_name', 'artist_name', 'popularity', 'track_album_id',
       'album_name', 'release_date', 'album_id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'}

only_in_set_1 = set_1 - set_2
only_in_set_2 = set_2 - set_1

print("Колонки, які є тільки у першому датасеті:")
print(only_in_set_1)

print("\nКолонки, які є тільки у другому датасеті:")
print(only_in_set_2)

Колонки, які є тільки у першому датасеті:
{'genres', 'artist_uri', 'artist_pop', 'time_signature', 'album_uri', 'track_uri'}

Колонки, які є тільки у другому датасеті:
{'track_id', 'track_album_id'}


In [55]:
to_merged_df = to_merged_df.rename(columns={'playlist_id': 'album_id'})
to_merged_df = to_merged_df.rename(columns={'track_popularity': 'popularity'})
to_merged_df = to_merged_df.rename(columns={'track_album_release_date': 'release_date'})
to_merged_df = to_merged_df.rename(columns={'track_album_name': 'album_name'})
to_merged_df = to_merged_df.rename(columns={'track_artist': 'artist_name'})
to_merged_df = to_merged_df.drop(columns=['playlist_name', 'playlist_subgenre', 'playlist_name', 'playlist_genre'])

In [14]:
# in our initial dataset, album_id is equal to playlist_id

In [56]:
common_columns = set_1 & set_2
print("Спільні колонки:", common_columns)


Спільні колонки: {'popularity', 'danceability', 'loudness', 'album_id', 'speechiness', 'instrumentalness', 'release_date', 'energy', 'liveness', 'acousticness', 'artist_name', 'track_name', 'tempo', 'album_name', 'key', 'duration_ms', 'mode', 'valence'}


In [22]:
def fetch_artist_details(artist_names):
    artist_data = []
    for name in tqdm(artist_names, desc="Fetching artist details"):
        try:
            results = sp.search(q=f'artist:{name}', type='artist', limit=1)
            if results['artists']['items']:
                artist = results['artists']['items'][0]
                data = {
                    'artist_name': name,
                    'artist_uri': artist['uri'],
                    'artist_pop': artist['popularity'],
                    'genres': ', '.join(artist['genres']) 
                }
            else:
                data = {
                    'artist_name': name,
                    'artist_uri': None,
                    'artist_pop': None,
                    'genres': None
                }
            artist_data.append(data)
        except Exception as e:
            with open("extract_log.txt", "a") as r:
                r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S") + ": " + str(e) + '\n')
            time.sleep(1)
            artist_data.append({
                'artist_name': name,
                'artist_uri': None,
                'artist_pop': None,
                'genres': None
            })
    return artist_data
artist_names = to_merged_df['artist_name'].unique().tolist()
all_artist_details = []
BATCH_SIZE = 25
for i in tqdm(range(0, len(artist_names), BATCH_SIZE), desc="Fetching artist details"):
    batch_names = artist_names[i:i + BATCH_SIZE]
    artist_details = fetch_artist_details(batch_names)
    all_artist_details.extend(artist_details)
artist_details_df = pd.DataFrame([t for t in all_artist_details if t is not None])
artist_details_df.to_csv('artist_details.csv', index=False)

artist_details_df.head()


Fetching artist details: 100%|██████████| 428/428 [22:26<00:00,  3.15s/it]


,artist_name,artist_uri,artist_pop,genres
0,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:artist:04gDigrS5kc9YWfZHwBETP,87.0,pop
2,Zara Larsson,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:artist:69GGBxA162lTqCwzJG5jLp,83.0,pop
4,Lewis Capaldi,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,81.0,"pop, uk pop"


Information about tracks we pulled out through Google Colab, here you can see code, and the data - 'track_details.csv'

In [57]:
to_merged_df.head()

,track_id,track_name,artist_name,popularity,track_album_id,album_name,release_date,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,37i9dQZF1DXcZDD7cfEKhW,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,37i9dQZF1DXcZDD7cfEKhW,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [58]:
to_merged_df['track_uri'] = 'spotify:track:' + to_merged_df['track_id']
to_merged_df[['track_id', 'track_uri']].head()


,track_id,track_uri
0,6f807x0ima9a1j3VPbc7VN,spotify:track:6f807x0ima9a1j3VPbc7VN
1,0r7CVbZTWZgbTCYdfa2P31,spotify:track:0r7CVbZTWZgbTCYdfa2P31
2,1z1Hg7Vb0AhHDiEmnDE79l,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l
3,75FpbthrwQmzHlBJLuGdC7,spotify:track:75FpbthrwQmzHlBJLuGdC7
4,1e8PAfcKUYoKkxPhrHqw4x,spotify:track:1e8PAfcKUYoKkxPhrHqw4x


In [59]:
to_merged_df['album_uri'] = 'spotify:album:' + to_merged_df['track_album_id']
to_merged_df[['track_album_id', 'album_uri']].head()

,track_album_id,album_uri
0,2oCs0DGTsRO98Gh5ZSl2Cx,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx
1,63rPSO264uRjW1X5E6cWv6,spotify:album:63rPSO264uRjW1X5E6cWv6
2,1HoSmj2eLcsrR0vE9gThr4,spotify:album:1HoSmj2eLcsrR0vE9gThr4
3,1nqYsOef1yKKuGOVchbsk6,spotify:album:1nqYsOef1yKKuGOVchbsk6
4,7m7vv9wlQ4i0LFuJiE2zsQ,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ


In [60]:
to_merged_df = to_merged_df.drop(columns=['track_album_id'])

In [61]:
to_merged_df.head()

,track_id,track_name,artist_name,popularity,album_name,release_date,album_id,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_uri,album_uri
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,6,...,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,11,...,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:album:63rPSO264uRjW1X5E6cWv6
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),2019-07-05,37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,1,...,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:album:1HoSmj2eLcsrR0vE9gThr4
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,2019-07-19,37i9dQZF1DXcZDD7cfEKhW,0.718,0.930,7,...,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,spotify:track:75FpbthrwQmzHlBJLuGdC7,spotify:album:1nqYsOef1yKKuGOVchbsk6
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),2019-03-05,37i9dQZF1DXcZDD7cfEKhW,0.650,0.833,1,...,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,spotify:track:1e8PAfcKUYoKkxPhrHqw4x,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ


In [62]:
artist_inf = pd.read_csv("artist_details.csv")
artist_inf

,artist_name,artist_uri,artist_pop,genres
0,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:artist:04gDigrS5kc9YWfZHwBETP,87.0,pop
2,Zara Larsson,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:artist:69GGBxA162lTqCwzJG5jLp,83.0,pop
4,Lewis Capaldi,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,81.0,"pop, uk pop"
...,...,...,...,...
10688,The Cataracs,spotify:artist:7C64wNX3howEFZjAYRKsfP,62.0,NaN
10689,Qubicon,spotify:artist:0BUPzXY1mPrEVDno3TGBwL,7.0,NaN
10690,Ferry Corsten feat. Jenny Wahlstrom,NaN,NaN,NaN
10691,Tegan and Sara,spotify:artist:5e1BZulIiYWPRm8yogwUYH,58.0,"canadian indie, lilith, metropopolis, permanen..."


In [63]:
merged_df = to_merged_df.merge(artist_inf, on='artist_name', how='left')
merged_df.head(2)

,track_id,track_name,artist_name,popularity,album_name,release_date,album_id,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,track_uri,album_uri,artist_uri,artist_pop,genres
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,6,...,0.00000,0.0653,0.518,122.036,194754,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,90.0,"pop, singer-songwriter pop, uk pop"
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,11,...,0.00421,0.3570,0.693,99.972,162600,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:album:63rPSO264uRjW1X5E6cWv6,spotify:artist:04gDigrS5kc9YWfZHwBETP,87.0,pop


In [64]:
df.columns

Index(['artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
       'duration_ms', 'album_name', 'album_id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'release_date', 'popularity', 'artist_pop', 'genres'],
      dtype='object')

In [65]:
merged_df = merged_df.drop(columns=['track_id'])

In [66]:
merged_df.columns

Index(['track_name', 'artist_name', 'popularity', 'album_name', 'release_date',
       'album_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'track_uri', 'album_uri',
       'artist_uri', 'artist_pop', 'genres'],
      dtype='object')

In [26]:
df.head(1)

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,0.904,0.813,...,0.0311,0.00697,0.0471,0.81,125.461,4.0,2005,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...


In [67]:
common_columns = [col for col in df.columns if col in merged_df.columns]
merged_df = merged_df[common_columns]
merged_df = merged_df.reindex(columns=df.columns)
merged_df.head(3)

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Ed Sheeran,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I Don't Care (with Justin Bieber) - Loud Luxur...,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,194754,I Don't Care (with Justin Bieber) [Loud Luxury...,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,...,0.1020,0.000000,0.0653,0.518,122.036,NaN,2019-06-14,66,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:artist:04gDigrS5kc9YWfZHwBETP,Memories - Dillon Francis Remix,spotify:album:63rPSO264uRjW1X5E6cWv6,162600,Memories (Dillon Francis Remix),37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,...,0.0724,0.004210,0.3570,0.693,99.972,NaN,2019-12-13,67,87.0,pop
2,Zara Larsson,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,All the Time - Don Diablo Remix,spotify:album:1HoSmj2eLcsrR0vE9gThr4,176616,All the Time (Don Diablo Remix),37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,...,0.0794,0.000023,0.1100,0.613,124.008,NaN,2019-07-05,70,77.0,"pop, scandipop, swedish electropop, swedish pop"


In [68]:
merged_df['release_date'] = merged_df['release_date'].astype(str).apply(lambda x: re.search(r'\d{4}', x).group(0) if re.search(r'\d{4}', x) else None)
merged_df['release_date']

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
32828    2014
32829    2013
32830    2014
32831    2014
32832    2014
Name: release_date, Length: 32833, dtype: object

In [69]:
songs_per_date = merged_df['release_date'].dropna().value_counts().sort_index()

print("Кількість пісень по унікальній даті:")
print(songs_per_date)

Кількість пісень по унікальній даті:
1957       2
1958       1
1960       4
1961       1
1962       2
        ... 
2016    2127
2017    2430
2018    3312
2019    9081
2020     785
Name: release_date, Length: 63, dtype: int64


In [70]:
merged_df['release_date'] = pd.to_numeric(merged_df['release_date'], errors='coerce')
merged_df = merged_df[(merged_df['release_date'] >= 2010) & (merged_df['release_date'] <= 2024)]
merged_df['release_date'] = merged_df['release_date'].astype('int16')
merged_df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Ed Sheeran,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I Don't Care (with Justin Bieber) - Loud Luxur...,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,194754,I Don't Care (with Justin Bieber) [Loud Luxury...,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,...,0.102000,0.000000,0.0653,0.5180,122.036,NaN,2019,66,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:artist:04gDigrS5kc9YWfZHwBETP,Memories - Dillon Francis Remix,spotify:album:63rPSO264uRjW1X5E6cWv6,162600,Memories (Dillon Francis Remix),37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,...,0.072400,0.004210,0.3570,0.6930,99.972,NaN,2019,67,87.0,pop
2,Zara Larsson,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,All the Time - Don Diablo Remix,spotify:album:1HoSmj2eLcsrR0vE9gThr4,176616,All the Time (Don Diablo Remix),37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,...,0.079400,0.000023,0.1100,0.6130,124.008,NaN,2019,70,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:track:75FpbthrwQmzHlBJLuGdC7,spotify:artist:69GGBxA162lTqCwzJG5jLp,Call You Mine - Keanu Silva Remix,spotify:album:1nqYsOef1yKKuGOVchbsk6,169093,Call You Mine - The Remixes,37i9dQZF1DXcZDD7cfEKhW,0.718,0.930,...,0.028700,0.000009,0.2040,0.2770,121.956,NaN,2019,60,83.0,pop
4,Lewis Capaldi,spotify:track:1e8PAfcKUYoKkxPhrHqw4x,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,Someone You Loved - Future Humans Remix,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ,189052,Someone You Loved (Future Humans Remix),37i9dQZF1DXcZDD7cfEKhW,0.650,0.833,...,0.080300,0.000000,0.0833,0.7250,123.976,NaN,2019,69,81.0,"pop, uk pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,Lush & Simon,spotify:track:7bxnKAamR3snQ1VGLuVfC1,spotify:artist:0UN8vGMxWEBKNNu8NtyF96,City Of Lights - Official Radio Edit,spotify:album:2azRoBBWEEEYhqV6sb7JrT,204375,City Of Lights (Vocal Mix),6jI1gFr6ANFtT8MmTvA2Ux,0.428,0.922,...,0.076600,0.000000,0.0668,0.2100,128.170,NaN,2014,42,37.0,"progressive electro house, sky room"
32829,Tegan and Sara,spotify:track:5Aevni09Em4575077nkWHz,spotify:artist:5e1BZulIiYWPRm8yogwUYH,Closer - Sultan & Ned Shepard Remix,spotify:album:6kD6KLxj7s8eCE3ABvAyf5,353120,Closer Remixed,6jI1gFr6ANFtT8MmTvA2Ux,0.522,0.786,...,0.001710,0.004270,0.3750,0.4000,128.041,NaN,2013,20,58.0,"canadian indie, lilith, metropopolis, permanen..."
32830,Starkillers,spotify:track:7ImMqPP3Q1yfUHvsdn7wEo,spotify:artist:3uYJQiv4uZCowxyUzcJj54,Sweet Surrender - Radio Edit,spotify:album:0ltWNSY9JgxoIZO4VzuCa6,210112,Sweet Surrender (Radio Edit),6jI1gFr6ANFtT8MmTvA2Ux,0.529,0.821,...,0.108000,0.000001,0.1500,0.4360,127.989,NaN,2014,14,26.0,russian punk rock
32831,Mat Zo,spotify:track:2m69mhnfQ1Oq6lGtXuYhgX,spotify:artist:2n7USVO8fO8FF8zq4kG2N1,Only For You - Maor Levi Remix,spotify:album:1fGrOkHnHJcStl14zNx8Jy,367432,Only For You (Remixes),6jI1gFr6ANFtT8MmTvA2Ux,0.626,0.888,...,0.007920,0.127000,0.3430,0.3080,128.008,NaN,2014,15,43.0,"complextro, edm, electro house, progressive house"


In [72]:
merged_df = merged_df[~merged_df['genres'].str.contains('russian', case=False, na=False)]
merged_df = merged_df[~merged_df['artist_name'].str.contains('Seryoga', case=False, na=False)]
merged_df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Ed Sheeran,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I Don't Care (with Justin Bieber) - Loud Luxur...,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,194754,I Don't Care (with Justin Bieber) [Loud Luxury...,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,...,0.102000,0.000000,0.0653,0.5180,122.036,NaN,2019,66,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:artist:04gDigrS5kc9YWfZHwBETP,Memories - Dillon Francis Remix,spotify:album:63rPSO264uRjW1X5E6cWv6,162600,Memories (Dillon Francis Remix),37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,...,0.072400,0.004210,0.3570,0.6930,99.972,NaN,2019,67,87.0,pop
2,Zara Larsson,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,All the Time - Don Diablo Remix,spotify:album:1HoSmj2eLcsrR0vE9gThr4,176616,All the Time (Don Diablo Remix),37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,...,0.079400,0.000023,0.1100,0.6130,124.008,NaN,2019,70,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:track:75FpbthrwQmzHlBJLuGdC7,spotify:artist:69GGBxA162lTqCwzJG5jLp,Call You Mine - Keanu Silva Remix,spotify:album:1nqYsOef1yKKuGOVchbsk6,169093,Call You Mine - The Remixes,37i9dQZF1DXcZDD7cfEKhW,0.718,0.930,...,0.028700,0.000009,0.2040,0.2770,121.956,NaN,2019,60,83.0,pop
4,Lewis Capaldi,spotify:track:1e8PAfcKUYoKkxPhrHqw4x,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,Someone You Loved - Future Humans Remix,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ,189052,Someone You Loved (Future Humans Remix),37i9dQZF1DXcZDD7cfEKhW,0.650,0.833,...,0.080300,0.000000,0.0833,0.7250,123.976,NaN,2019,69,81.0,"pop, uk pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,Ferry Corsten feat. Jenny Wahlstrom,spotify:track:0aBDrRTgDCwWbcOnEIp7DJ,NaN,Many Ways - Radio Edit,spotify:album:59XOfNjuYZB6feC6QUzS3e,196993,Many Ways,6jI1gFr6ANFtT8MmTvA2Ux,0.581,0.640,...,0.026600,0.000000,0.5720,0.2880,128.001,NaN,2013,27,NaN,NaN
32828,Lush & Simon,spotify:track:7bxnKAamR3snQ1VGLuVfC1,spotify:artist:0UN8vGMxWEBKNNu8NtyF96,City Of Lights - Official Radio Edit,spotify:album:2azRoBBWEEEYhqV6sb7JrT,204375,City Of Lights (Vocal Mix),6jI1gFr6ANFtT8MmTvA2Ux,0.428,0.922,...,0.076600,0.000000,0.0668,0.2100,128.170,NaN,2014,42,37.0,"progressive electro house, sky room"
32829,Tegan and Sara,spotify:track:5Aevni09Em4575077nkWHz,spotify:artist:5e1BZulIiYWPRm8yogwUYH,Closer - Sultan & Ned Shepard Remix,spotify:album:6kD6KLxj7s8eCE3ABvAyf5,353120,Closer Remixed,6jI1gFr6ANFtT8MmTvA2Ux,0.522,0.786,...,0.001710,0.004270,0.3750,0.4000,128.041,NaN,2013,20,58.0,"canadian indie, lilith, metropopolis, permanen..."
32831,Mat Zo,spotify:track:2m69mhnfQ1Oq6lGtXuYhgX,spotify:artist:2n7USVO8fO8FF8zq4kG2N1,Only For You - Maor Levi Remix,spotify:album:1fGrOkHnHJcStl14zNx8Jy,367432,Only For You (Remixes),6jI1gFr6ANFtT8MmTvA2Ux,0.626,0.888,...,0.007920,0.127000,0.3430,0.3080,128.008,NaN,2014,15,43.0,"complextro, edm, electro house, progressive house"


In [73]:
def contains_cyrillic(text):
    return bool(re.search(r'[а-яА-ЯёЁ]', str(text)))
russian_songs = merged_df[merged_df['track_name'].apply(contains_cyrillic)]
merged_df = merged_df[~merged_df['track_name'].apply(contains_cyrillic)]
merged_df = merged_df[~merged_df['artist_name'].apply(contains_cyrillic)]
merged_df = merged_df[~merged_df['album_name'].apply(contains_cyrillic)]
merged_df.shape

(23678, 24)

In [74]:
keywords = ['Russian', 'Russia', 'Moscow', 'Soviet', 'Putin', 'USSR', 'Siberia', 'belarusian', 'Seryoga']
pattern = '|'.join(keywords)
merged_df = merged_df[~merged_df.astype(str).apply(lambda x: x.str.contains(pattern, case=False, na=False)).any(axis=1)]
print(merged_df.shape)


(23670, 24)


In [75]:
merged_df = merged_df.drop(merged_df[merged_df['artist_name'] == 'Seryoga'].index)
print(merged_df.head())


        artist_name                             track_uri  \
0        Ed Sheeran  spotify:track:6f807x0ima9a1j3VPbc7VN   
1          Maroon 5  spotify:track:0r7CVbZTWZgbTCYdfa2P31   
2      Zara Larsson  spotify:track:1z1Hg7Vb0AhHDiEmnDE79l   
3  The Chainsmokers  spotify:track:75FpbthrwQmzHlBJLuGdC7   
4     Lewis Capaldi  spotify:track:1e8PAfcKUYoKkxPhrHqw4x   

                              artist_uri  \
0  spotify:artist:6eUKZXaKkcviH0Ku9w2n3V   
1  spotify:artist:04gDigrS5kc9YWfZHwBETP   
2  spotify:artist:1Xylc3o4UrD53lo9CvFvVg   
3  spotify:artist:69GGBxA162lTqCwzJG5jLp   
4  spotify:artist:4GNC7GD6oZMSxPGyXy4MNB   

                                          track_name  \
0  I Don't Care (with Justin Bieber) - Loud Luxur...   
1                    Memories - Dillon Francis Remix   
2                    All the Time - Don Diablo Remix   
3                  Call You Mine - Keanu Silva Remix   
4            Someone You Loved - Future Humans Remix   

                              a

In [76]:
found_rows = merged_df[merged_df['artist_name'] == 'Seryoga']
print(found_rows)


Empty DataFrame
Columns: [artist_name, track_uri, artist_uri, track_name, album_uri, duration_ms, album_name, album_id, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, release_date, popularity, artist_pop, genres]
Index: []

[0 rows x 24 columns]


In [77]:
merged_df

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Ed Sheeran,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I Don't Care (with Justin Bieber) - Loud Luxur...,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,194754,I Don't Care (with Justin Bieber) [Loud Luxury...,37i9dQZF1DXcZDD7cfEKhW,0.748,0.916,...,0.102000,0.000000,0.0653,0.5180,122.036,NaN,2019,66,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:artist:04gDigrS5kc9YWfZHwBETP,Memories - Dillon Francis Remix,spotify:album:63rPSO264uRjW1X5E6cWv6,162600,Memories (Dillon Francis Remix),37i9dQZF1DXcZDD7cfEKhW,0.726,0.815,...,0.072400,0.004210,0.3570,0.6930,99.972,NaN,2019,67,87.0,pop
2,Zara Larsson,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,All the Time - Don Diablo Remix,spotify:album:1HoSmj2eLcsrR0vE9gThr4,176616,All the Time (Don Diablo Remix),37i9dQZF1DXcZDD7cfEKhW,0.675,0.931,...,0.079400,0.000023,0.1100,0.6130,124.008,NaN,2019,70,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:track:75FpbthrwQmzHlBJLuGdC7,spotify:artist:69GGBxA162lTqCwzJG5jLp,Call You Mine - Keanu Silva Remix,spotify:album:1nqYsOef1yKKuGOVchbsk6,169093,Call You Mine - The Remixes,37i9dQZF1DXcZDD7cfEKhW,0.718,0.930,...,0.028700,0.000009,0.2040,0.2770,121.956,NaN,2019,60,83.0,pop
4,Lewis Capaldi,spotify:track:1e8PAfcKUYoKkxPhrHqw4x,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,Someone You Loved - Future Humans Remix,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ,189052,Someone You Loved (Future Humans Remix),37i9dQZF1DXcZDD7cfEKhW,0.650,0.833,...,0.080300,0.000000,0.0833,0.7250,123.976,NaN,2019,69,81.0,"pop, uk pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,Ferry Corsten feat. Jenny Wahlstrom,spotify:track:0aBDrRTgDCwWbcOnEIp7DJ,NaN,Many Ways - Radio Edit,spotify:album:59XOfNjuYZB6feC6QUzS3e,196993,Many Ways,6jI1gFr6ANFtT8MmTvA2Ux,0.581,0.640,...,0.026600,0.000000,0.5720,0.2880,128.001,NaN,2013,27,NaN,NaN
32828,Lush & Simon,spotify:track:7bxnKAamR3snQ1VGLuVfC1,spotify:artist:0UN8vGMxWEBKNNu8NtyF96,City Of Lights - Official Radio Edit,spotify:album:2azRoBBWEEEYhqV6sb7JrT,204375,City Of Lights (Vocal Mix),6jI1gFr6ANFtT8MmTvA2Ux,0.428,0.922,...,0.076600,0.000000,0.0668,0.2100,128.170,NaN,2014,42,37.0,"progressive electro house, sky room"
32829,Tegan and Sara,spotify:track:5Aevni09Em4575077nkWHz,spotify:artist:5e1BZulIiYWPRm8yogwUYH,Closer - Sultan & Ned Shepard Remix,spotify:album:6kD6KLxj7s8eCE3ABvAyf5,353120,Closer Remixed,6jI1gFr6ANFtT8MmTvA2Ux,0.522,0.786,...,0.001710,0.004270,0.3750,0.4000,128.041,NaN,2013,20,58.0,"canadian indie, lilith, metropopolis, permanen..."
32831,Mat Zo,spotify:track:2m69mhnfQ1Oq6lGtXuYhgX,spotify:artist:2n7USVO8fO8FF8zq4kG2N1,Only For You - Maor Levi Remix,spotify:album:1fGrOkHnHJcStl14zNx8Jy,367432,Only For You (Remixes),6jI1gFr6ANFtT8MmTvA2Ux,0.626,0.888,...,0.007920,0.127000,0.3430,0.3080,128.008,NaN,2014,15,43.0,"complextro, edm, electro house, progressive house"


In [78]:
song_count = merged_df.groupby('album_id').size().reset_index(name='song_count')
song_count_dict = song_count.set_index('album_id')['song_count'].to_dict()
song_count_dict

{'0275i1VNfBnsNbPl0QIBpG': 49,
 '03qQtbNHoJuFezRu2CnLuF': 1,
 '03sDEv7FN58Mb9CJOs1Tgn': 44,
 '06zrBJ5cts5aemZmqe80J7': 46,
 '07SNJ4MwYba9wwmzrbjmYi': 77,
 '07zF8MjQPsiYUXiAIGZ5TA': 13,
 '08QTrfsYYouffgnPjmllAQ': 99,
 '0AFYmoSuoMQiGGjzvBwr6u': 98,
 '0Ar0Ng9DlAWZtSPBvOQgOa': 4,
 '0B2HdP15IucgE0vk3sluJR': 41,
 '0BwUQpqHSlC2YfKwOp2dQV': 29,
 '0C2zyPdlkbWHrwVg9gqNdU': 2,
 '0COM4nZqdXC3sMjVdcaU7V': 50,
 '0DIwi1dMI59NItQtiNcevy': 31,
 '0DdGqNa18DwYyfIR05OrW1': 36,
 '0EmGNFBBgqLmvdQGPbgiMp': 4,
 '0FCHg9zJMNNiOokh3hVcxd': 94,
 '0HD4Pc1PK8fsyKQq9e2U2v': 46,
 '0Hr2h94pKN8QAGVAgD6BsD': 80,
 '0JmBB9HfrzDiZoPVRdv8ns': 100,
 '0Jw0HckkxCfIrOvpN081eV': 1,
 '0KNLLunbFh9XgitPMOGmQg': 5,
 '0MhTMIo1bgH6zzPh7BdChT': 95,
 '0NCspsyf0OS4BsPgGhkQXM': 100,
 '0QN4FeJQ1mpCygRg9r2JIK': 17,
 '0QUlq0LMDVXSBtbyYLTyZh': 11,
 '0SqaMfNsngZCpPw0UuelA7': 47,
 '0T47pLnihK4xB8Bk9IQ0Dv': 45,
 '0TT57Pe3RZNGCy98G1UQpM': 8,
 '0UXwwVDipbBQeEX7h4YuKU': 90,
 '0VS9w0NY4KXfLORkhY81s8': 1,
 '0VVH2Nzj6kBVGK3WIUQMAw': 9,
 '0VsVywryrqLhH

In [79]:
unique_ids = {album_id: idx for idx, album_id in enumerate(song_count_dict.keys(), start=1000000)}
merged_df['album_id'] = merged_df['album_id'].map(unique_ids)
song_count_dict = {unique_ids[key]: value for key, value in song_count_dict.items()}


print(list(song_count_dict.items())[:5])
merged_df


[(1000000, 49), (1000001, 1), (1000002, 44), (1000003, 46), (1000004, 77)]


,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,album_id,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,Ed Sheeran,spotify:track:6f807x0ima9a1j3VPbc7VN,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,I Don't Care (with Justin Bieber) - Loud Luxur...,spotify:album:2oCs0DGTsRO98Gh5ZSl2Cx,194754,I Don't Care (with Justin Bieber) [Loud Luxury...,1000229,0.748,0.916,...,0.102000,0.000000,0.0653,0.5180,122.036,NaN,2019,66,90.0,"pop, singer-songwriter pop, uk pop"
1,Maroon 5,spotify:track:0r7CVbZTWZgbTCYdfa2P31,spotify:artist:04gDigrS5kc9YWfZHwBETP,Memories - Dillon Francis Remix,spotify:album:63rPSO264uRjW1X5E6cWv6,162600,Memories (Dillon Francis Remix),1000229,0.726,0.815,...,0.072400,0.004210,0.3570,0.6930,99.972,NaN,2019,67,87.0,pop
2,Zara Larsson,spotify:track:1z1Hg7Vb0AhHDiEmnDE79l,spotify:artist:1Xylc3o4UrD53lo9CvFvVg,All the Time - Don Diablo Remix,spotify:album:1HoSmj2eLcsrR0vE9gThr4,176616,All the Time (Don Diablo Remix),1000229,0.675,0.931,...,0.079400,0.000023,0.1100,0.6130,124.008,NaN,2019,70,77.0,"pop, scandipop, swedish electropop, swedish pop"
3,The Chainsmokers,spotify:track:75FpbthrwQmzHlBJLuGdC7,spotify:artist:69GGBxA162lTqCwzJG5jLp,Call You Mine - Keanu Silva Remix,spotify:album:1nqYsOef1yKKuGOVchbsk6,169093,Call You Mine - The Remixes,1000229,0.718,0.930,...,0.028700,0.000009,0.2040,0.2770,121.956,NaN,2019,60,83.0,pop
4,Lewis Capaldi,spotify:track:1e8PAfcKUYoKkxPhrHqw4x,spotify:artist:4GNC7GD6oZMSxPGyXy4MNB,Someone You Loved - Future Humans Remix,spotify:album:7m7vv9wlQ4i0LFuJiE2zsQ,189052,Someone You Loved (Future Humans Remix),1000229,0.650,0.833,...,0.080300,0.000000,0.0833,0.7250,123.976,NaN,2019,69,81.0,"pop, uk pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,Ferry Corsten feat. Jenny Wahlstrom,spotify:track:0aBDrRTgDCwWbcOnEIp7DJ,NaN,Many Ways - Radio Edit,spotify:album:59XOfNjuYZB6feC6QUzS3e,196993,Many Ways,1000407,0.581,0.640,...,0.026600,0.000000,0.5720,0.2880,128.001,NaN,2013,27,NaN,NaN
32828,Lush & Simon,spotify:track:7bxnKAamR3snQ1VGLuVfC1,spotify:artist:0UN8vGMxWEBKNNu8NtyF96,City Of Lights - Official Radio Edit,spotify:album:2azRoBBWEEEYhqV6sb7JrT,204375,City Of Lights (Vocal Mix),1000407,0.428,0.922,...,0.076600,0.000000,0.0668,0.2100,128.170,NaN,2014,42,37.0,"progressive electro house, sky room"
32829,Tegan and Sara,spotify:track:5Aevni09Em4575077nkWHz,spotify:artist:5e1BZulIiYWPRm8yogwUYH,Closer - Sultan & Ned Shepard Remix,spotify:album:6kD6KLxj7s8eCE3ABvAyf5,353120,Closer Remixed,1000407,0.522,0.786,...,0.001710,0.004270,0.3750,0.4000,128.041,NaN,2013,20,58.0,"canadian indie, lilith, metropopolis, permanen..."
32831,Mat Zo,spotify:track:2m69mhnfQ1Oq6lGtXuYhgX,spotify:artist:2n7USVO8fO8FF8zq4kG2N1,Only For You - Maor Levi Remix,spotify:album:1fGrOkHnHJcStl14zNx8Jy,367432,Only For You (Remixes),1000407,0.626,0.888,...,0.007920,0.127000,0.3430,0.3080,128.008,NaN,2014,15,43.0,"complextro, edm, electro house, progressive house"


In [47]:
combined_df = pd.concat([df, merged_df], ignore_index=True)
combined_df.shape

(2205203, 24)

In [80]:
combined_df = combined_df.dropna(subset=['artist_name', 'artist_uri'])
combined_df['genres'] = combined_df['genres'].fillna('unknown')
mean_time_signature = combined_df['time_signature'].mean()
combined_df['time_signature'] = combined_df['time_signature'].fillna(mean_time_signature)
combined_df.isnull().sum()

artist_name         0
track_uri           0
artist_uri          0
track_name          0
album_uri           0
duration_ms         0
album_name          0
album_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
release_date        0
popularity          0
artist_pop          0
genres              0
dtype: int64

In [81]:
combined_df.to_csv("final_dataset.csv", index=False)

In [84]:
df = pd.read_csv("final_dataset.csv")

In [85]:
df.isnull().sum()

artist_name         0
track_uri           0
artist_uri          0
track_name          0
album_uri           0
duration_ms         0
album_name          0
album_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
release_date        0
popularity          0
artist_pop          0
genres              0
dtype: int64

In [86]:
found_rows = df[df['artist_name'] == 'Seryoga']
print(found_rows)

        artist_name                             track_uri  \
417433      Seryoga  spotify:track:7psGkZlVISuCkpkVF1vD5R   
544858      Seryoga  spotify:track:02U7tu5WRzrzUPgWMAxiFK   
1212123     Seryoga  spotify:track:2NvK91RDMKrVka3K0KVqhc   
1212124     Seryoga  spotify:track:3fYrtvfwXsk2gY7t7tT333   
1212125     Seryoga  spotify:track:13InssmjAjm526zWRved9L   
1212126     Seryoga  spotify:track:0gCMLnl9d4NI1yOqNxJamo   
1212128     Seryoga  spotify:track:4RvzIxL70XrtSNr0mkVLFv   
1212129     Seryoga  spotify:track:36MQiI770WNFEUr1IRHWlO   
1212130     Seryoga  spotify:track:7lT7hgsFfWc0NgVakf1iHo   
2095326     Seryoga  spotify:track:1kU7R70RdjUqZyKTmWuqFr   
2112018     Seryoga  spotify:track:7ACBqsaKK4tkkLUkThrGgK   

                                    artist_uri  \
417433   spotify:artist:00nBviCFsh0Cpf8zp0sxw3   
544858   spotify:artist:00nBviCFsh0Cpf8zp0sxw3   
1212123  spotify:artist:00nBviCFsh0Cpf8zp0sxw3   
1212124  spotify:artist:00nBviCFsh0Cpf8zp0sxw3   
1212125  spotify:

In [88]:
df = df.drop(merged_df[df['artist_name'] == 'Seryoga'].index)
print(df.shape)

/var/folders/nd/7lptb8w57_g497n2mc741rmc0000gn/T/ipykernel_1007/1876320278.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(merged_df[df['artist_name'] == 'Seryoga'].index)


(2205039, 24)


In [89]:
keywords = ['Russian', 'Russia', 'Moscow', 'Soviet', 'Putin', 'USSR', 'Siberia', 'belarusian', 'Seryoga']
pattern = '|'.join(keywords)
df = df[~df.astype(str).apply(lambda x: x.str.contains(pattern, case=False, na=False)).any(axis=1)]
print(df.shape)


(2204951, 24)


In [90]:
song_count = df.groupby('album_id').size().reset_index(name='song_count')
song_count_dict = song_count.set_index('album_id')['song_count'].to_dict()
song_count_dict

{0: 51,
 1: 39,
 2: 62,
 3: 125,
 4: 17,
 5: 72,
 6: 16,
 7: 52,
 8: 46,
 9: 20,
 10: 72,
 11: 128,
 12: 9,
 13: 13,
 14: 95,
 15: 7,
 16: 102,
 17: 73,
 18: 67,
 19: 83,
 20: 14,
 21: 14,
 22: 39,
 23: 20,
 24: 7,
 25: 38,
 26: 24,
 27: 11,
 28: 75,
 29: 31,
 30: 145,
 31: 91,
 32: 94,
 33: 16,
 34: 68,
 35: 55,
 36: 9,
 37: 67,
 38: 40,
 39: 55,
 40: 56,
 41: 184,
 42: 116,
 43: 17,
 44: 28,
 45: 49,
 46: 11,
 47: 14,
 48: 20,
 49: 45,
 50: 35,
 51: 33,
 52: 13,
 53: 93,
 54: 5,
 55: 108,
 56: 18,
 57: 19,
 58: 37,
 59: 111,
 60: 53,
 61: 6,
 62: 20,
 63: 32,
 64: 86,
 65: 109,
 66: 74,
 67: 8,
 68: 152,
 69: 16,
 70: 69,
 71: 12,
 72: 189,
 73: 23,
 74: 5,
 75: 13,
 76: 25,
 77: 21,
 78: 14,
 79: 16,
 80: 51,
 81: 34,
 82: 25,
 83: 52,
 84: 28,
 85: 35,
 86: 26,
 87: 24,
 88: 58,
 89: 85,
 90: 95,
 91: 58,
 92: 6,
 93: 40,
 94: 81,
 95: 133,
 96: 18,
 97: 224,
 98: 76,
 99: 42,
 100: 21,
 101: 18,
 102: 40,
 103: 159,
 104: 49,
 105: 70,
 106: 98,
 107: 22,
 108: 49,
 109: 124,
 110

In [91]:
df = df[~df['track_name'].apply(contains_cyrillic)]
df = df[~df['artist_name'].apply(contains_cyrillic)]
df = df[~df['album_name'].apply(contains_cyrillic)]

In [92]:
df.shape

(2204863, 24)

In [94]:
df.to_csv("final.csv", index=False)